## Metrics

汇总常见2分类的指标，例如: AUC，ROC曲线，ACC, 敏感性， 特异性，精确度，召回率，PPV, NPV, F1

具体的介绍，可以参考一下：https://blog.csdn.net/sunflower_sara/article/details/81214897

In [ ]:
import os
from datetime import datetime
from onekey_algo import get_param_in_cwd

os.makedirs('img', exist_ok=True)
os.makedirs('results', exist_ok=True)
# 模型日志位置，如果没有更改默认保存位置，并且模型是当天训练出来的，可以不动这个参数。
model_root = get_param_in_cwd('model_root', 'models')

In [ ]:
import pandas as pd

samples = pd.read_csv('group.csv')
samples['ID'] = samples['ID'].astype(str)
samples

In [ ]:
import pandas as pd
import numpy  as np
from onekey_algo.custom.components import metrics
from onekey_algo.custom.components.comp1 import draw_roc
from matplotlib import pyplot as plt

# log_path 修改为Onekey val目录中对应的log文件。
metric_results = []
all_predict_scores = []
all_gts = []
epoch_mapping = {'resnet50': 11, 'resnet101': 11, 'densenet121': 11, 'inception_v3': 10, 'vgg19': 10}
for model in ['resnet50', 'resnet18', 'inception_v3']:
    all_pred = []
    all_gt = []
    for subset in ['Train', 'Test']:
        cohort = 'TRAIN' if subset == 'Train' else 'VAL'
        log_path = os.path.join(model_root, model, f"viz/BST_{cohort}_RESULTS.txt")
#         log_path = os.path.join(model_root, model, f"{cohort.lower()}/Epoch-{epoch_mapping[model]}.txt")
        val_log = pd.read_csv(log_path, names=['fname', 'pred_score', 'pred_label', 'gt'], sep='\t')
        val_log['ID'] = val_log['fname'].map(lambda x: os.path.basename(os.path.dirname(x)))
        val_log = pd.merge(val_log, samples, on='ID', how='inner')
        print(val_log.shape)
        ul_labels = np.unique(val_log['pred_label'])
        for ul in [1]:
            pred_score = list(map(lambda x: x[0] if x[1] == ul else 1-x[0], np.array(val_log[['pred_score', 'pred_label']])))
            gt = [1 if gt_ == ul else 0 for gt_ in np.array(val_log['gt'])]
            acc, auc, ci, tpr, tnr, ppv, npv, precision, recall, f1, thres = metrics.analysis_pred_binary(gt, pred_score,
                                                                                                          use_youden=True)
            ci = f"{ci[0]:.4f}-{ci[1]:.4f}"
            metric_results.append([model, acc, auc, ci, tpr, tnr, ppv, npv, precision, recall, f1, thres, subset])
            all_pred.append(np.array(list(map(lambda x: (1-x[0], x[0]) if x[1] == 1 else (x[0], 1-x[0]), 
                                          np.array(val_log[['pred_score', 'pred_label']])))))
            all_gt.append(gt)
    all_predict_scores.extend(all_pred)
    all_gts.extend(all_gt)
    draw_roc(all_gt, all_pred, labels=['Train', 'Val'], title=f"Model: {model}")
    plt.savefig(f'img/DTL_{model}_roc.svg', bbox_inches='tight')
    plt.show()
metrics = pd.DataFrame(metric_results, 
             columns=['ModelName', 'Acc', 'AUC', '95% CI', 'Sensitivity', 'Specificity', 'PPV', 'NPV', 
                      'Precision', 'Recall', 'F1', 'Threshold', 'Cohort'])
metrics

# 保存预测结果

将深度学习的预测结果，保存中与组学的预测结果相同的格式，f1便后续进行汇总。

In [ ]:
for model in os.listdir(model_root):
    log_path = os.path.join(model_root, model, "viz/BST_{subset}_RESULTS.txt")
    train_ = pd.read_csv(log_path.format(subset='TRAIN'), names=['fname', 'pred_score', 'pred_label', 'gt'], sep='\t')
    train_['group'] = 'train'
    val_ = pd.read_csv(log_path.format(subset='VAL'), names=['fname', 'pred_score', 'pred_label', 'gt'], sep='\t')
    val_['group'] = 'test'
    data = pd.concat([train_, val_], axis=0)

    predict = []
    for idx, row in data.iterrows():
        predict.append([row['fname'].replace('.png', '.gz'), 
                        row['pred_score'] if row['pred_label'] == 0 else 1 - row['pred_score'],
                        row['pred_score'] if row['pred_label'] == 1 else 1 - row['pred_score'],
                        row['group']])
    predict = pd.DataFrame(predict, columns=['ID', 'label-0', 'label-1', 'group'])
    predict[predict['group'] == 'train'][['ID', 'label-0', 'label-1']].to_csv(f'results/DTL_{model}_train.csv', index=False)
    predict[predict['group'] == 'test'][['ID', 'label-0', 'label-1']].to_csv(f'results/DTL_{model}_test.csv', index=False)
    predict